<a href="https://colab.research.google.com/github/wooohun/BERT-Summarizer/blob/main/COMP_541.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import pandas as pd

In [2]:
# install kaggle
!pip install -q kaggle
!mkdir ~/.kaggle

# move kaggle api token to kaggle folder
!cp -v kaggle.json ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists
'kaggle.json' -> '/root/.kaggle/kaggle.json'


In [3]:
# download dataset
!kaggle datasets download -d gowrishankarp/newspaper-text-summarization-cnn-dailymail
!unzip newspaper-text-summarization-cnn-dailymail

newspaper-text-summarization-cnn-dailymail.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  newspaper-text-summarization-cnn-dailymail.zip
replace cnn_dailymail/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

In [5]:
# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

In [6]:
BERT_MODEL = 'uncased_L-12_H-768_A-12' # uncased Base BERT model
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

In [21]:
tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)
test_tokens = tokenizer.tokenize("This here's an example of using the BERT tokenizer")
test_token_ids = tokenizer.convert_tokens_to_ids(test_tokens)
test_tokens

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [43]:
# transfer datasets to DataFrame

test = pd.read_csv("cnn_dailymail/test.csv")
train = pd.read_csv("cnn_dailymail/train.csv")
validate = pd.read_csv("cnn_dailymail/validation.csv")

In [44]:
# tokenize articles, store in same row as article
test['article_tokens'] = test['article'].map(lambda x: tokenizer.tokenize(x))

In [45]:
# get lengths of article token list for filtering by # of tokens
test['article_token_len'] = test['article_tokens'].map(lambda x: len(x))

In [46]:
test

,id,article,highlights,article_tokens,article_token_len
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...,"[ever, noticed, how, plane, seats, appear, to,...",430
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...,"[a, drunk, teenage, boy, had, to, be, rescued,...",388
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...,"[doug, ##ie, freed, ##man, is, on, the, verge,...",133
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...,"[liverpool, target, net, ##o, is, also, wanted...",397
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6...","[bruce, jen, ##ner, will, break, his, silence,...",970
...,...,...,...,...,...
11485,ed8674cc15b29a87d8df8de1efee353d71122272,Our young Earth may have collided with a body ...,Oxford scientists say a Mercury-like body stru...,"[our, young, earth, may, have, collided, with,...",797
11486,2f58d1a99e9c47914e4b1c31613e3a041cd9011e,A man facing trial for helping his former love...,Man accused of helping former lover kill woman...,"[a, man, facing, trial, for, helping, his, for...",991
11487,411f6d57825161c3a037b4742baccd6cd227c0c3,A dozen or more metal implements are arranged ...,Marianne Power tried the tuning fork facial at...,"[a, dozen, or, more, metal, implements, are, a...",1819
11488,b5683ef8342056b17b068e0d59bdbe87e3fe44ea,Brook Lopez dominated twin brother Robin with ...,Brooklyn Nets beat the Portland Trail Blazers ...,"[brook, lopez, dominated, twin, brother, robin...",232
